# Wikipedia Data Merging

In this Jupyter Notebook, we verify the data webscraped from Wikipedia (see the [data](./data) folder). We'll check the individual .csv files and make sure we can merged them in a main datafile.

## Section 0 - Setting up the environment
Importing libraries, files and define functions

In [1]:
import pandas as pd

In [2]:
df_happiness = pd.read_csv('./data/happiness.csv')
df_happiness.set_index('countries', inplace = True);
df_happiness.drop(df_happiness.columns[0], axis=1, inplace = True);
# Our initial DataFrame, we'll left-inner-join the other data source to this one,
# index following the 'countries' column (we didn't know about the 'index_col=0' option at the time =) 

In [3]:
country_rename_dict = {"China, People's Republic of" : "China",
                       "Cote d'Ivoire": "Ivory Coast",
                       "Côte d'Ivoire": "Ivory Coast",
                       "People's Republic of China": "China",
                       'Burma': 'Myanmar',
                       'Congo': 'Congo (Brazzaville)',
                       'Congo, Democratic Republic of the': 'Congo (Kinshasa)',
                       'Congo, Republic of the': 'Congo (Brazzaville)',
                       'Congo, Republic of': 'Congo (Brazzaville)',
                       'Czech Republic': 'Czechia',
                       'Democratic Republic of Congo': 'Congo (Kinshasa)',
                       'Democratic Republic of the Congo': 'Congo (Kinshasa)',
                       'DR Congo': 'Congo (Kinshasa)',
                       'Eswatini (Swaziland)': 'Eswatini',
                       'Gambia, The': 'Gambia',
                       'Great Britain': 'United Kingdom',
                       'Korea, North': 'North Korea',
                       'Korea, South': 'South Korea',
                       'Republic of the Congo': 'Congo (Brazzaville)',
                       'Swaziland': 'Eswatini',
                       'The Gambia' : 'Gambia',
                       'Trinidad and Tobago': 'Trinidad & Tobago',
                       'United States': 'United States of America'
                      }
# this dictionary will be used to bring the different country name styles to a common denominator

In [4]:
def rename_country(country_name):
    """
    Renames the country name to the default spelling, as in the happiness DataFrame. 
    
    Arguments: 
    country_name -- name of the country to check and see if it's an alternative spelling
    
    Returns: 
    default spelling of country name (could be the same as input)
    """
    
    if country_name in country_rename_dict.keys():
        return country_rename_dict[country_name]
    else:
        return country_name

In [5]:
def make_mergeable(df):
    """
    Prepares a DataFrame to be merged with the happiness DataFrame.
    
    Arguments:
    df -- DataFrame to be prepared, assumed to contain a 'countries' column
    
    Returns:
    df -- the modified DataFrame, with countries changed to default spelling, 
    index set to countries column
    """
    
    df['countries'] = df['countries'].apply(lambda x: rename_country(x));
    # renames countries in line with the happiness index

    df.set_index('countries', inplace = True);
    df.drop(df.columns[0], axis=1, inplace = True);
    # resets index to countries
    
    return df

In [6]:
def merge_with_happiness(df):
    """
    Left inner join to the happiness DataFrame
    
    Arguments:
    df --DataFrame to merge with happiness
    
    Returns:
    happiness DataFrame with the added data issued from 'df'
    """
    
    return df_happiness.join(df, how = 'left')

In [7]:
def merge_and_check_empty(csv_filename, new_column_label):
    """
    Recurring function used to check the elements which are in the happiness DataFrame
    but not in the new DataFrame we want to merge it with. 
    
    Arguments: 
    csv_filename -- filename of the file we want to merge with happiness
    new_column_label -- column in the new DataFrame we will check for empty data
    
    Returns: 
    the combined DataFrame filtered down to rows where the new DataFrame had 
    missing elements
    """
    
    df = pd.read_csv(csv_filename)
    df = make_mergeable(df)
    df_combined = df_happiness.join(df, how = 'left')
    return df_combined[df_combined[new_column_label].isnull()]

## Section 1 - Individual Data Merging Analysis

In this section, we merge the different .csv files with the happiness file by countries, and check for missing data. We used this process to build up the *country_rename_dict*, the dictionary containing the alternative and default spellings of specific countries. 

### alcohol_consumption.csv

In [8]:
merge_and_check_empty('./data/alcohol_consumption.csv', 'alcohol_consumption')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,alcohol_consumption
countries,,,,,,,,
Taiwan,6.446,1.368,1.430,0.914,0.351,0.242,0.097,NaN
Kosovo,6.100,0.882,1.232,0.758,0.489,0.262,0.006,NaN
Northern Cyprus,5.718,1.263,1.252,1.042,0.417,0.191,0.162,NaN
Hong Kong,5.430,1.438,1.277,1.122,0.440,0.258,0.287,NaN
Palestinian Territories,4.696,0.657,1.247,0.672,0.225,0.103,0.066,NaN
South Sudan,2.853,0.306,0.575,0.295,0.010,0.202,0.091,NaN


### cigarettes.csv

In [9]:
merge_and_check_empty('./data/cigarettes.csv', 'cigarette_consumption')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,cigarette_consumption
countries,,,,,,,,
United Arab Emirates,6.825,1.503,1.310,0.825,0.598,0.262,0.182,NaN
Taiwan,6.446,1.368,1.430,0.914,0.351,0.242,0.097,NaN
Kosovo,6.100,0.882,1.232,0.758,0.489,0.262,0.006,NaN
Northern Cyprus,5.718,1.263,1.252,1.042,0.417,0.191,0.162,NaN
Hong Kong,5.430,1.438,1.277,1.122,0.440,0.258,0.287,NaN
Bhutan,5.082,0.813,1.321,0.604,0.457,0.370,0.167,NaN
Palestinian Territories,4.696,0.657,1.247,0.672,0.225,0.103,0.066,NaN


### economic_sectors.csv

In [10]:
merge_and_check_empty('./data/economical_sectors.csv', 'agricultural')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,agricultural,industrial,service
countries,,,,,,,,,,
Northern Cyprus,5.718,1.263,1.252,1.042,0.417,0.191,0.162,NaN,NaN,NaN
Palestinian Territories,4.696,0.657,1.247,0.672,0.225,0.103,0.066,NaN,NaN,NaN
South Sudan,2.853,0.306,0.575,0.295,0.010,0.202,0.091,NaN,NaN,NaN


### government_system.csv

In [11]:
merge_and_check_empty('./data/government_system.csv', 'const_form')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,const_form,head_of_state
countries,,,,,,,,,
Taiwan,6.446,1.368,1.430,0.914,0.351,0.242,0.097,NaN,NaN
Kosovo,6.100,0.882,1.232,0.758,0.489,0.262,0.006,NaN,NaN
Northern Cyprus,5.718,1.263,1.252,1.042,0.417,0.191,0.162,NaN,NaN
Libya,5.525,1.044,1.303,0.673,0.416,0.133,0.152,NaN,NaN
Hong Kong,5.430,1.438,1.277,1.122,0.440,0.258,0.287,NaN,NaN
Palestinian Territories,4.696,0.657,1.247,0.672,0.225,0.103,0.066,NaN,NaN
Yemen,3.380,0.287,1.163,0.463,0.143,0.108,0.077,NaN,NaN


Libya and Yemen are listed with an empty value, the other countries are not in this *government_system* list

### median_age.csv

In [12]:
merge_and_check_empty('./data/median_age.csv', 'median_age')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,median_age,median_age_male,median_age_female
countries,,,,,,,,,,
Northern Cyprus,5.718,1.263,1.252,1.042,0.417,0.191,0.162,NaN,NaN,NaN
Palestinian Territories,4.696,0.657,1.247,0.672,0.225,0.103,0.066,NaN,NaN,NaN


### olympic_medals_2016.csv

In [13]:
merge_and_check_empty('./data/olympic_medals_2016.csv', 'gold_medals_2016')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,gold_medals_2016,silver_medals_2016,bronze_medals_2016
countries,,,,,,,,,,
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,NaN,NaN,NaN
Costa Rica,7.167,1.034,1.441,0.963,0.558,0.144,0.093,NaN,NaN,NaN
Luxembourg,7.090,1.609,1.479,1.012,0.526,0.194,0.316,NaN,NaN,NaN
Malta,6.726,1.300,1.520,0.999,0.564,0.375,0.151,NaN,NaN,NaN
Taiwan,6.446,1.368,1.430,0.914,0.351,0.242,0.097,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411,NaN,NaN,NaN
Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147,NaN,NaN,NaN
Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025,NaN,NaN,NaN


We will assume these empty values to be 0 in the [WB notebook](WB.ipynb)

### oscar.csv

In [14]:
merge_and_check_empty('./data/oscar.csv', 'oscar_winning')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,oscar_winning
countries,,,,,,,,
United States of America,6.892,1.433,1.457,0.874,0.454,0.280,0.128,NaN
United Arab Emirates,6.825,1.503,1.310,0.825,0.598,0.262,0.182,NaN
Qatar,6.374,1.684,1.313,0.871,0.555,0.220,0.167,NaN
El Salvador,6.253,0.794,1.242,0.789,0.430,0.093,0.074,NaN
Bahrain,6.199,1.362,1.368,0.871,0.536,0.255,0.110,NaN
Trinidad & Tobago,6.192,1.231,1.477,0.713,0.489,0.185,0.016,NaN
Cyprus,6.046,1.263,1.223,1.042,0.406,0.190,0.041,NaN
Jamaica,5.890,0.831,1.478,0.831,0.490,0.107,0.028,NaN
Mauritius,5.888,1.120,1.402,0.798,0.498,0.215,0.060,NaN


Again, we will assume these empty values to be 0 in the [WB notebook](WB.ipynb)

### railways.csv

In [15]:
merge_and_check_empty('./data/railways.csv', 'railways_per_capita')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,railways_per_capita
countries,,,,,,,,
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,NaN
Malta,6.726,1.300,1.520,0.999,0.564,0.375,0.151,NaN
Qatar,6.374,1.684,1.313,0.871,0.555,0.220,0.167,NaN
Bahrain,6.199,1.362,1.368,0.871,0.536,0.255,0.110,NaN
Trinidad & Tobago,6.192,1.231,1.477,0.713,0.489,0.185,0.016,NaN
Nicaragua,6.105,0.694,1.325,0.835,0.435,0.200,0.127,NaN
Kosovo,6.100,0.882,1.232,0.758,0.489,0.262,0.006,NaN
Cyprus,6.046,1.263,1.223,1.042,0.406,0.190,0.041,NaN
Kuwait,6.021,1.500,1.319,0.808,0.493,0.142,0.097,NaN


Actually missing from railways: Kosovo, Northern Cyprus, Congo (Brazzaville), Palestinian Territories, Comoros, Central African Republic, South Sudan. The rest is confirmed to have 0 railways. (see  in the [WB notebook](WB.ipynb))

### lifeexp.csv

In [16]:
merge_and_check_empty('./data/lifeexp.csv', 'lifeexps')

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,lifeexps
countries,,,,,,,,
Taiwan,6.446,1.368,1.430,0.914,0.351,0.242,0.097,NaN
Qatar,6.374,1.684,1.313,0.871,0.555,0.220,0.167,NaN
Kosovo,6.100,0.882,1.232,0.758,0.489,0.262,0.006,NaN
Estonia,5.893,1.237,1.528,0.874,0.495,0.103,0.161,NaN
Northern Cyprus,5.718,1.263,1.252,1.042,0.417,0.191,0.162,NaN
Albania,4.719,0.947,0.848,0.874,0.383,0.178,0.027,NaN
Palestinian Territories,4.696,0.657,1.247,0.672,0.225,0.103,0.066,NaN
Somalia,4.668,0.000,0.698,0.268,0.559,0.243,0.270,NaN


Those countries don't have life expectancy data on Wikipedia

## Section 2 - Merging All

In this section, we are merging all the csv files with the happiness file, and save everything in one combined csv. 

In [17]:
list_of_csv_files = ['alcohol_consumption',
                     'cigarettes',
                     'economical_sectors',
                     'government_system',
                     'lifeexp',
                     'median_age',
                     'olympic_medals_2016',
                     'oscar',
                     'railways'
                    ]

In [18]:
df_combined = df_happiness.copy()

In [19]:
for file in list_of_csv_files:
    csv_filename = './data/' + file + '.csv'
    df = pd.read_csv(csv_filename)
    df = make_mergeable(df)
    df_combined = df_combined.join(df, how = 'left')

In [20]:
df_combined.head()

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,alcohol_consumption,cigarette_consumption,agricultural,...,head_of_state,lifeexps,median_age,median_age_male,median_age_female,gold_medals_2016,silver_medals_2016,bronze_medals_2016,oscar_winning,railways_per_capita
countries,,,,,,,,,,,,,,,,,,,,,
Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,12.3,1098.8,2.7%,...,Ceremonial,81.7,42.5,40.9,44.3,0.0,0.0,1.0,0.0,929
Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,11.4,1298.0,1.3%,...,Ceremonial,80.8,42.2,41.2,43.2,2.0,6.0,7.0,3.0,"2,893"
Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,7.7,552.8,2.3%,...,Ceremonial,82.3,39.2,38.4,40.0,0.0,0.0,4.0,0.0,"1,350"
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,7.1,848.1,5.8%,...,Ceremonial,82.9,36.5,35.9,37.1,NaN,NaN,NaN,0.0,NaN
Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,9.9,1459.9,1.6%,...,Ceremonial,82.1,42.6,41.5,43.6,8.0,7.0,4.0,3.0,"5,591"


In [21]:
df_combined.tail()

,happiness,GDP_per_capita,social_support,healthy_life_exp,freedom,generosity,corruption,alcohol_consumption,cigarette_consumption,agricultural,...,head_of_state,lifeexps,median_age,median_age_male,median_age_female,gold_medals_2016,silver_medals_2016,bronze_medals_2016,oscar_winning,railways_per_capita
countries,,,,,,,,,,,,,,,,,,,,,
Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411,9.8,94.0,30.9%,...,Executive,68.7,19.0,18.3,19.8,NaN,NaN,NaN,NaN,NaN
Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147,7.7,181.8,23.4%,...,Executive,65.0,17.7,17.5,18.0,NaN,NaN,NaN,0.0,"15,866"
Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025,0.7,311.6,23.0%,...,Executive,64.5,18.9,18.8,18.9,NaN,NaN,NaN,0.0,"418,827"
Central African Republic,3.083,0.026,0.000,0.105,0.225,0.235,0.035,3.8,213.2,43.2%,...,Executive,52.8,19.7,19.4,20.0,NaN,NaN,NaN,NaN,NaN
South Sudan,2.853,0.306,0.575,0.295,0.010,0.202,0.091,NaN,383.7,NaN,...,Executive,57.6,17.3,17.2,17.5,NaN,NaN,NaN,NaN,NaN


And as a final step, we are saving the combined DataFrame as a csv. 

In [22]:
df_combined.to_csv('./data/combined_df.csv')